In [20]:
import requests

# Caudoputamen connectivity data
caudoputamenUrl = "https://bams2.bams1.org/search/connections/inputs/?page={number}&query=Caudoputamen+~+%28CP%29"
htmls = []

with open('BAMS_data/connectivity/get_requests.txt', 'wb') as file:
        file.write(b'')

for i in range(1, 18):
    url = caudoputamenUrl.replace("{number}", str(i))
    res = requests.get(url)
    htmls.append(res.content)
    print("getting", i)
    with open('BAMS_data/connectivity/get_requests.txt', 'ab') as file:
        file.write(res.content)






getting 1
getting 2
getting 3
getting 4
getting 5
getting 6
getting 7
getting 8
getting 9
getting 10
getting 11
getting 12
getting 13
getting 14
getting 15
getting 16
getting 17


In [57]:
from bs4 import BeautifulSoup
import csv
import re

htmls = []
with open('BAMS_data/connectivity/get_requests.html', 'r') as file:
        htmls = file.read().split("<!DOCTYPE")

htmls = list(filter(len, htmls))
 
with open('BAMS_data/connectivity/caudoputamen_inputs.csv', 'w') as csvfile:
    csvfile.write("")
    
add_header = True    

output_rows = []
for html in htmls:
    soup = BeautifulSoup(html)
    table = soup.find("table")
    
    
    for table_row in table.findAll('tr'):
        if add_header:
            headers = table_row.findAll('th')
            output_row = []
            for header in headers:
                output_row.append(header.text.replace("\n", "").strip())
            output_rows.append(output_row)
            add_header = False
            
        columns = table_row.findAll('td')
        output_row = []
        for column in columns:
            column_text = column.text.replace("\n", "").strip()
            column_text = re.sub(' +', ' ', column_text)
            output_row.append(column_text)
        if len(output_row) > 0:
            output_rows.append(output_row)
    
with open('BAMS_data/connectivity/caudoputamen_inputs.csv', 'a') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(output_rows)
    
print("Tables converted to csv")
    

Tables converted to csv
